# LSTM With doc2Vec

Transform each basket into a vector, then train a LSTM on the previous baskets to predict the next basket

In [1]:

import os
import collections

import random
import sqlite3
import numpy as np

# Load Data

In [2]:
def getOrders(lowerlimit, upperlimit):
    
    conn = sqlite3.connect("instacart.db")
    cur = conn.cursor()
    
    # Get final order
    cur.execute("SELECT B.user_id as user_id, A.order_id as order_id, "
                " B.order_number as order_number, A.product_id as product_id "
                "FROM products_train A INNER JOIN orders B "
                " ON A.order_id = B.order_id "
                "WHERE A.order_id % 100 >= " + str(lowerlimit) + \
                " AND A.order_id % 100 <= " + str(upperlimit) + ";")
    train_order = np.array(cur.fetchall())
    
    # Get all prior orders
    cur.execute( \
        "SELECT D.user_id as user_id, "
        "  D.order_id as order_id, "
        "  D.order_number as order_number, "
        "  C.product_id as product_id "
        "FROM products_prior C INNER JOIN ( "
        "  SELECT DISTINCT A.user_id as user_id,"
        "    A.order_id as order_id, A.order_number as order_number "
        "  FROM orders A INNER JOIN ( "
        "    SELECT DISTINCT user_id FROM orders "
        "    WHERE eval_set = 'train' "
        "      AND order_id % 100 >= " + str(lowerlimit) + 
        "      AND order_id % 100 <= " + str(upperlimit) +
        "    ) B ON A.user_id = B.user_id WHERE A.eval_set = 'prior' "
        ") D ON C.order_id = D.order_id;")
    prior_orders = np.array(cur.fetchall())
    
    conn.close()
    
    return train_order, prior_orders

In [10]:
y, x = getOrders(0, 69)

print(x.shape)
print(y.shape)

(14458765, 4)
(968968, 4)


# Convert purchases to OHE

In [ ]:

order_dict = dict()

for i in range(x.shape[0]):
    try:
        order_dict[x[i][1]][x[i][2]-1] = 1
    except:
        order_dict[x[i][1]] = np.zeros((49688))
        order_dict[x[i][1]][x[i][2]-1] = 1      

# Train LSTM Model with Basket Vectors

In [12]:
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

# create and fit the LSTM network
LSTM_model = Sequential()
LSTM_model.add(LSTM(100, stateful=True, batch_input_shape=(100, 5, 49688))) # input_shape=(batchsize, timesteps, data_dim)
LSTM_model.add(Dense(49688, activation='sigmoid'))
LSTM_model.compile(loss='binary_crossentropy', optimizer='adam')

## Create training data in batches of 500 orders

In [14]:
curPosition = 0

for n in range(100):
    print("Starting batch " + str(n) + " of ten.")
    print("--------------------------------")
    # Delete x_train / y_train from last iteration, may fail if this is first time through
    try:
        del y_train
    except:
        pass

    try:
        del x_train
    except:
        pass

    
    
    # i iterates over 500 orders 
    for i in range(100):
        
        # Print status
        if i % 25 == 0:
            print ("Working on order " + str(i) + " of this batch...")
        
        # New order id coming in:
        current_order_id = y[curPosition][1]
        current_user_id = y[curPosition][0]
        current_user_final_order_number = y[curPosition][2]
        
        # Initialize One Hot Encoding, one zero for each product
        current_user_y = np.zeros((49688))
        
        # This loop increments curPosition at the end   
        while current_order_id == y[curPosition][1]:
            
            # make the position in current_order_y = 1 for each product_id
            # Zero index is product_id == 1, so subtract 1 from the product id
            current_user_y[y[curPosition][3] - 1] = 1
        
            curPosition += 1
        

        # current_order_y is done, let's make the current_order_x: 
        # For reference: x and y have format: [user_id, order_id, order_number, product_id]

        
        # grab this user's prior orders
        # Only grab orders where the order number is in the last five orders
        prior_orders_products = x[np.where((x[:,0] == current_user_id) & (x[:,2] >= current_user_final_order_number - 5))]

        # initialize current_user_x
        current_user_x = np.zeros((5, 49688))

        # for each product in prior_orders_products, add to current_user_x
        for p in prior_orders_products:
            
            # current_user_x[a][b], where... 
            # a = timestep = The timestep is order_number - (current_user_final_order_number + 5)
            # b = product_id in OHE, minus 1 because of zero indexing
            current_user_x[p[2] - (current_user_final_order_number - 5)][p[3] - 1] = 1
    
        # Assert - current_user_x is done, add to x_train and y_train

        # Expand full order to batch dimension
        cur_user_x_train = np.expand_dims(current_user_x, axis=0)

        # Create x_train or add to it 
        try: 
            x_train = np.append(x_train, cur_user_x_train, axis=0)
        except:
            x_train = cur_user_x_train
            
        # do the same for y: 
        # Expand full order to batch dimension
        cur_user_y_train = np.expand_dims(current_user_y, axis=0)

        # Create x_train or add to it 
        try: 
            y_train = np.append(y_train, cur_user_y_train, axis=0)
        except:
            y_train = cur_user_y_train

    # Done creating batch, time to train
    LSTM_model.fit(x_train, y_train, epochs=15, batch_size=x_train.shape[0], verbose=2)
    

Starting batch 0 of ten.
--------------------------------
Working on order 0 of this batch...
Working on order 25 of this batch...
Working on order 50 of this batch...
Working on order 75 of this batch...
Epoch 1/15
1s - loss: 0.6845
Epoch 2/15
1s - loss: 0.6805
Epoch 3/15
1s - loss: 0.6748
Epoch 4/15
1s - loss: 0.6664
Epoch 5/15
1s - loss: 0.6544
Epoch 6/15
1s - loss: 0.6373
Epoch 7/15
1s - loss: 0.6135
Epoch 8/15
1s - loss: 0.5816
Epoch 9/15
1s - loss: 0.5421
Epoch 10/15
1s - loss: 0.4991
Epoch 11/15
1s - loss: 0.4560
Epoch 12/15
1s - loss: 0.4163
Epoch 13/15
1s - loss: 0.3806
Epoch 14/15
1s - loss: 0.3474
Epoch 15/15
1s - loss: 0.3155
Starting batch 1 of ten.
--------------------------------
Working on order 0 of this batch...
Working on order 25 of this batch...
Working on order 50 of this batch...
Working on order 75 of this batch...
Epoch 1/15
1s - loss: 0.2927
Epoch 2/15
1s - loss: 0.2633
Epoch 3/15
1s - loss: 0.2349
Epoch 4/15
1s - loss: 0.2075
Epoch 5/15
1s - loss: 0.1813
Epoc

KeyboardInterrupt: 

In [9]:
LSTM_model.fit(x_train, y_train, epochs=15, batch_size=x_train.shape[0], verbose=2)


/opt/conda/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:2252: UserWarning: Expected no kwargs, you passed 1
kwargs passed to function are ignored with Tensorflow backend
  warnings.warn('\n'.join(msg))


Epoch 1/15
7s - loss: 0.6932
Epoch 2/15
5s - loss: 0.6927
Epoch 3/15
5s - loss: 0.6921
Epoch 4/15
5s - loss: 0.6916
Epoch 5/15
5s - loss: 0.6909
Epoch 6/15
5s - loss: 0.6899
Epoch 7/15
5s - loss: 0.6886
Epoch 8/15
5s - loss: 0.6867
Epoch 9/15
5s - loss: 0.6838
Epoch 10/15
5s - loss: 0.6795
Epoch 11/15
5s - loss: 0.6733
Epoch 12/15
5s - loss: 0.6641
Epoch 13/15
5s - loss: 0.6508
Epoch 14/15
5s - loss: 0.6317
Epoch 15/15
5s - loss: 0.6050


# Score the validation orders

## Grab validation orders:

In [100]:
# Get validation orders:
y_validation, x_validation = getOrders(70, 75)

## Change LSTM Model to accept one order at a time

In [16]:
# re-define model
LSTM_model_predict = Sequential()
LSTM_model_predict.add(LSTM(100, stateful=True, batch_input_shape=(1, 5, 49688))) # input_shape=(batchsize, timesteps, data_dim)
LSTM_model_predict.add(Dense(49688, activation='sigmoid'))

# copy weights
old_weights = LSTM_model.get_weights()
LSTM_model_predict.set_weights(old_weights)

# compile model
LSTM_model_predict.compile(loss='mean_squared_error', optimizer='adam')


## Make Function to predict for a given order

In [22]:
# New order id coming in:
current_order_id = y[curPosition][1]
current_user_id = y[curPosition][0]
current_user_final_order_number = y[curPosition][2]

# Initialize One Hot Encoding, one zero for each product
current_user_y = np.zeros((49688))


[ 0.  0.  0. ...,  0.  0.  0.]


In [23]:
# New order id coming in:
current_order_id = y[curPosition][1]
current_user_id = y[curPosition][0]
current_user_final_order_number = y[curPosition][2]

# Initialize One Hot Encoding, one zero for each product
current_user_y = np.zeros((49688))

# This loop increments curPosition at the end   
while current_order_id == y[curPosition][1]:

    # make the position in current_order_y = 1 for each product_id
    # Zero index is product_id == 1, so subtract 1 from the product id
    current_user_y[y[curPosition][3] - 1] = 1

    curPosition += 1


# current_order_y is done, let's make the current_order_x: 
# For reference: x and y have format: [user_id, order_id, order_number, product_id]


# grab this user's prior orders
# Only grab orders where the order number is in the last five orders
prior_orders_products = x[np.where((x[:,0] == current_user_id) & (x[:,2] >= current_user_final_order_number - 5))]

# initialize current_user_x
current_user_x = np.zeros((5, 49688))

# for each product in prior_orders_products, add to current_user_x
for p in prior_orders_products:

    # current_user_x[a][b], where... 
    # a = timestep = The timestep is order_number - (current_user_final_order_number + 5)
    # b = product_id in OHE, minus 1 because of zero indexing
    current_user_x[p[2] - (current_user_final_order_number - 5)][p[3] - 1] = 1

# Assert - current_user_x is done, add to x_train and y_train

# Expand full order to batch dimension
cur_user_x_train = np.expand_dims(current_user_x, axis=0)

x_train = cur_user_x_train





In [25]:
print(x_train.shape)
print(current_user_y.shape)

(1, 5, 49688)
(49688,)


In [26]:
# Make Prediction
prediction = LSTM_model_predict.predict(x_train)

/opt/conda/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:2252: UserWarning: Expected no kwargs, you passed 1
kwargs passed to function are ignored with Tensorflow backend
  warnings.warn('\n'.join(msg))


In [42]:
for i in range(prediction.shape[1]):
    print(prediction[0][i])

0.11657
0.119506
0.117374
0.118505
0.118588
0.117461
0.118109
0.118284
0.116774
0.11927
0.118848
0.11752
0.118542
0.118171
0.120582
0.116813
0.117879
0.117738
0.118059
0.11934
0.11929
0.119547
0.11989
0.117938
0.127695
0.119712
0.116924
0.118927
0.118477
0.11603
0.119629
0.117449
0.118633
0.127416
0.117345
0.119677
0.119413
0.116572
0.117993
0.11845
0.118525
0.119258
0.117828
0.119131
0.134943
0.118048
0.118096
0.117556
0.121177
0.118853
0.117617
0.11853
0.11877
0.128821
0.117745
0.117797
0.118507
0.119713
0.118334
0.117262
0.117994
0.118726
0.120586
0.119629
0.116983
0.119777
0.11676
0.119495
0.116769
0.120941
0.120484
0.118638
0.118651
0.118656
0.118521
0.11985
0.120186
0.115822
0.123662
0.117527
0.118782
0.118226
0.117837
0.120677
0.11808
0.119341
0.116885
0.118961
0.118322
0.117201
0.117464
0.118605
0.121088
0.117538
0.126917
0.11901
0.118259
0.117613
0.118464
0.116812
0.11746
0.119507
0.119129
0.11832
0.118867
0.11697
0.120262
0.118873
0.12091
0.117732
0.11645
0.118737
0.119465
0.

0.133714
0.118974
0.117131
0.117979
0.118057
0.119144
0.11789
0.1185
0.118307
0.118278
0.119357
0.117146
0.117591
0.117917
0.11876
0.118468
0.11913
0.11877
0.117861
0.11806
0.117582
0.118841
0.117779
0.11875
0.118817
0.117325
0.116882
0.118919
0.118504
0.118692
0.11935
0.118958
0.122716
0.120127
0.117306
0.116399
0.117789
0.120664
0.11873
0.119016
0.118991
0.118647
0.117287
0.123775
0.126405
0.118346
0.119783
0.119116
0.117402
0.117794
0.118774
0.11896
0.116658
0.118369
0.118174
0.11642
0.117306
0.118265
0.117758
0.121166
0.118975
0.119259
0.120609
0.118027
0.117062
0.116812
0.119439
0.119083
0.118577
0.118043
0.11848
0.118249
0.120512
0.117763
0.117742
0.119804
0.119121
0.120658
0.117148
0.118336
0.118956
0.117177
0.117575
0.119268
0.118747
0.118637
0.117566
0.11839
0.11768
0.119978
0.11889
0.117162
0.118133
0.118528
0.11858
0.117353
0.118055
0.119263
0.118279
0.118315
0.118572
0.118492
0.116149
0.117958
0.138529
0.119076
0.118448
0.118686
0.118985
0.121017
0.120845
0.117368
0.119015


0.120593
0.116743
0.118332
0.119298
0.117659
0.118348
0.119006
0.117061
0.118518
0.118367
0.128258
0.117219
0.118373
0.117149
0.119344
0.118969
0.118951
0.117557
0.119773
0.118917
0.118114
0.118277
0.117947
0.116874
0.11838
0.118152
0.119139
0.119337
0.119312
0.117568
0.120325
0.119375
0.128267
0.119778
0.11809
0.119162
0.117758
0.116638
0.116451
0.120968
0.127437
0.119442
0.119176
0.118839
0.118081
0.117867
0.119161
0.118769
0.11734
0.118053
0.118095
0.119302
0.120444
0.118632
0.119831
0.118253
0.119082
0.119801
0.119547
0.120137
0.118969
0.118169
0.118359
0.118593
0.119512
0.119555
0.118667
0.119588
0.119158
0.118973
0.117791
0.119062
0.123091
0.119585
0.117826
0.117279
0.117294
0.117271
0.119041
0.117046
0.11787
0.117402
0.117146
0.118892
0.120008
0.116204
0.118929
0.119426
0.117621
0.118241
0.118536
0.117556
0.118386
0.120375
0.118178
0.118541
0.116389
0.119594
0.117962
0.118483
0.128611
0.119177
0.11833
0.120421
0.117601
0.118484
0.119215
0.118443
0.119328
0.118694
0.118218
0.1192

0.122148
0.117336
0.119416
0.117318
0.119166
0.118035
0.119128
0.11809
0.118116
0.120428
0.120477
0.117549
0.121296
0.119461
0.116962
0.11772
0.118058
0.118971
0.117795
0.154729
0.118371
0.122967
0.118038
0.116778
0.116453
0.118352
0.116856
0.128483
0.117065
0.11824
0.120056
0.123948
0.119076
0.119761
0.118176
0.117096
0.121822
0.118337
0.118568
0.124575
0.116323
0.118206
0.121475
0.117635
0.116919
0.118433
0.118765
0.117218
0.11617
0.121399
0.11737
0.119355
0.120219
0.126299
0.120536
0.119859
0.118756
0.118318
0.116678
0.118034
0.117599
0.118795
0.119127
0.119767
0.117217
0.118163
0.119325
0.117731
0.116856
0.118062
0.1189
0.117231
0.119834
0.118291
0.118859
0.117175
0.118555
0.117485
0.1192
0.124478
0.120512
0.118911
0.119091
0.119544
0.118246
0.117792
0.116016
0.119664
0.118528
0.119163
0.120305
0.120132
0.120998
0.117197
0.118176
0.118384
0.117322
0.1175
0.118026
0.118898
0.118121
0.117055
0.1196
0.117949
0.118667
0.118967
0.117403
0.119586
0.116997
0.117396
0.117635
0.118028
0.116

0.119743
0.127894
0.115914
0.120934
0.116692
0.118832
0.119375
0.117963
0.116819
0.117943
0.117952
0.121498
0.119127
0.119697
0.118375
0.11856
0.119708
0.119896
0.11955
0.118588
0.118605
0.117994
0.118897
0.119392
0.119032
0.116479
0.117693
0.117892
0.119923
0.117918
0.118493
0.126318
0.118533
0.117777
0.118212
0.118451
0.117592
0.116847
0.118874
0.116982
0.127622
0.119348
0.11799
0.119502
0.118265
0.119283
0.119567
0.120099
0.120061
0.119212
0.11812
0.119556
0.116073
0.118361
0.117898
0.117922
0.118657
0.119471
0.119367
0.118231
0.118822
0.118267
0.119803
0.119116
0.118134
0.117737
0.120645
0.119166
0.118373
0.116745
0.118095
0.117055
0.11709
0.1182
0.117971
0.117487
0.118261
0.119634
0.118715
0.119786
0.120658
0.117765
0.11898
0.117157
0.117224
0.118053
0.117772
0.116711
0.118108
0.116911
0.118379
0.121801
0.119567
0.125778
0.117012
0.125993
0.117085
0.120058
0.139249
0.119279
0.118083
0.117902
0.117426
0.117656
0.120239
0.117083
0.118885
0.118116
0.130094
0.117284
0.117036
0.121202


...

0.118182
0.119166
0.116807
0.119377
0.118878
0.117016
0.11895
0.118691
0.11732
0.1176
0.118468
0.119745
0.118146
0.118946
0.119968
0.119409
0.118872
0.12018
0.117474
0.118742
0.120588
0.118962
0.116409
0.116883
0.120941
0.116423
0.119755
0.118683
0.119391
0.118552
0.119074
0.118289
0.118704
0.117279
0.125061
0.127863
0.117604
0.131053
0.118705
0.117215
0.119362
0.11607
0.118505
0.118867
0.11963
0.119667
0.11727
0.11895
0.120346
0.119654
0.11777
0.123541
0.11644
0.11747
0.117242
0.119057
0.119904
0.119514
0.119321
0.118623
0.117889
0.118317
0.117883
0.118595
0.119259
0.117388
0.119598
0.117447
0.119972
0.118362
0.117672
0.118167
0.115318
0.119261
0.118874
0.118412
0.118213
0.117817
0.117603
0.123799
0.118535
0.120262
0.118018
0.118648
0.119976
0.119423
0.118937
0.117202
0.120256
0.11732
0.117875
0.118732
0.118573
0.119509
0.117102
0.117447
0.124684
0.117265
0.119585
0.118683
0.119608
0.118944
0.118437
0.119875
0.117091
0.119603
0.11892
0.118536
0.11719
0.117983
0.118308
0.117269
0.11964

0.117837
0.119516
0.120272
0.118594
0.12547
0.131529
0.122601
0.119492
0.119789
0.118981
0.118338
0.120538
0.117575
0.116258
0.119245
0.118929
0.127651
0.126249
0.117015
0.121018
0.117487
0.119197
0.11704
0.119104
0.118043
0.11963
0.143331
0.121464
0.125265
0.117787
0.119264
0.117264
0.121904
0.128969
0.11932
0.120575
0.120739
0.119556
0.11828
0.12886
0.118282
0.116128
0.117217
0.118352
0.118184
0.118233
0.123159
0.119006
0.119656
0.116349
0.118994
0.118132
0.117594
0.118514
0.118638
0.119137
0.117581
0.125292
0.117602
0.117669
0.119801
0.119219
0.1248
0.119276
0.117401
0.118113
0.117605
0.119106
0.118289
0.116596
0.119096
0.118077
0.118756
0.120466
0.116546
0.118965
0.118883
0.119047
0.117807
0.119166
0.119831
0.170121
0.116102
0.119452
0.118461
0.150516
0.118744
0.116208
0.119724
0.117943
0.117829
0.116913
0.116317
0.120401
0.119703
0.118682
0.119115
0.119199
0.118671
0.117669
0.116701
0.119263
0.160856
0.11952
0.117391
0.120196
0.118512
0.118543
0.117702
0.118457
0.118312
0.118743
0

0.118374
0.119197
0.119441
0.117075
0.118472
0.117853
0.118099
0.11482
0.119065
0.117248
0.119303
0.119688
0.123138
0.118708
0.118751
0.118251
0.123082
0.119004
0.117862
0.11563
0.119824
0.117464
0.121731
0.11741
0.118558
0.117165
0.122363
0.119374
0.11846
0.119552
0.126563
0.119576
0.117224
0.116814
0.119797
0.119526
0.117315
0.117937
0.118522
0.118091
0.118329
0.118329
0.11914
0.11732
0.119267
0.126032
0.117141
0.11767
0.116355
0.148529
0.118005
0.118891
0.117693
0.118219
0.11684
0.118473
0.118883
0.116108
0.119329
0.133031
0.119685
0.124035
0.119727
0.119803
0.116997
0.121106
0.117063
0.118566
0.131137
0.119075
0.119706
0.117911
0.118874
0.119581
0.118667
0.117747
0.118585
0.118162
0.118572
0.126477
0.121013
0.117795
0.117382
0.118087
0.12082
0.118403
0.120668
0.11686
0.120094
0.118147
0.117955
0.117167
0.124141
0.120681
0.117458
0.119031
0.117753
0.118688
0.117915
0.118855
0.11902
0.120787
0.117317
0.119843
0.119387
0.118555
0.117842
0.119261
0.117265
0.118682
0.118411
0.117031
0.1

0.118765
0.119518
0.117899
0.118788
0.121466
0.118635
0.117351
0.118984
0.117603
0.118661
0.118013
0.118994
0.119689
0.12083
0.118686
0.119289
0.117749
0.118033
0.118892
0.121991
0.118031
0.120566
0.117979
0.118083
0.118204
0.119056
0.116986
0.117675
0.116461
0.118484
0.118028
0.123932
0.120731
0.116096
0.118073
0.119703
0.119057
0.118775
0.116972
0.119368
0.117704
0.119335
0.119348
0.117695
0.119518
0.120258
0.117528
0.120313
0.117963
0.119666
0.119404
0.116704
0.11859
0.116871
0.117018
0.118326
0.116983
0.117343
0.117283
0.130162
0.117637
0.118643
0.117239
0.118045
0.119022
0.118855
0.117326
0.118899
0.11766
0.118824
0.118086
0.125956
0.119345
0.118211
0.119786
0.116816
0.118877
0.117704
0.117549
0.116642
0.117537
0.117156
0.11859
0.118431
0.119579
0.116955
0.11681
0.116149
0.118731
0.119017
0.118415
0.116319
0.116503
0.119812
0.118152
0.119994
0.123419
0.118049
0.11816
0.119313
0.118527
0.117735
0.12076
0.119592
0.118716
0.119204
0.121974
0.11861
0.118458
0.118389
0.119046
0.119897


0.116576
0.118121
0.118428
0.117776
0.118756
0.119487
0.117744
0.118266
0.118869
0.118328
0.124169
0.118652
0.118764
0.119007
0.118045
0.118349
0.117275
0.121439
0.118445
0.119095
0.116594
0.117598
0.119015
0.117438
0.118752
0.118972
0.116484
0.118085
0.116702
0.119403
0.118255
0.118512
0.117717
0.120482
0.11807
0.119876
0.137815
0.116407
0.134319
0.119691
0.118813
0.11826
0.119642
0.118859
0.116915
0.119813
0.116862
0.11793
0.119043
0.116737
0.119683
0.118528
0.118552
0.119111
0.118462
0.118232
0.125308
0.117737
0.11899
0.119695
0.116775
0.124276
0.118043
0.118869
0.118387
0.118633
0.116986
0.117492
0.117935
0.117232
0.118363
0.117617
0.117298
0.126018
0.118906
0.117999
0.117204
0.117009
0.117817
0.122056
0.11857
0.117681
0.117596
0.119034
0.124156
0.117841
0.119521
0.118868
0.118984
0.115783
0.119138
0.116841
0.119264
0.11781
0.120001
0.12103
0.118386
0.117918
0.119363
0.12014
0.118532
0.120123
0.121574
0.118928
0.117255
0.11858
0.118139
0.11868
0.11796
0.116166
0.117602
0.117778
0.1

In [53]:
for i in range(current_user_y.shape[0]):
    if current_user_y[i] > 0:
        print("product_id is " + str(i))
        print("The prediction was " + str(prediction[0][i]))


product_id is 1515
The prediction was 0.12261
product_id is 8276
The prediction was 0.183793
product_id is 13175
The prediction was 0.291524
product_id is 27965
The prediction was 0.216771
product_id is 33231
The prediction was 0.12317
product_id is 36871
The prediction was 0.118982
product_id is 47143
The prediction was 0.153239
product_id is 47629
The prediction was 0.185216
product_id is 47765
The prediction was 0.243661
product_id is 48449
The prediction was 0.119319


In [54]:
for i in range(prediction.shape[1]):
    if prediction[0][i] > .2:
        print("product_id = " + str(i) + " - the prediction was " + str(prediction[0][i]))

product_id = 195 - the prediction was 0.201058
product_id = 4604 - the prediction was 0.242964
product_id = 4919 - the prediction was 0.210356
product_id = 5449 - the prediction was 0.216713
product_id = 5875 - the prediction was 0.221926
product_id = 8423 - the prediction was 0.23494
product_id = 8517 - the prediction was 0.231792
product_id = 9075 - the prediction was 0.21066
product_id = 13175 - the prediction was 0.291524
product_id = 15289 - the prediction was 0.203092
product_id = 16796 - the prediction was 0.237196
product_id = 17793 - the prediction was 0.224791
product_id = 19056 - the prediction was 0.250771
product_id = 21136 - the prediction was 0.276767
product_id = 21615 - the prediction was 0.219575
product_id = 21902 - the prediction was 0.309531
product_id = 21937 - the prediction was 0.22144
product_id = 22934 - the prediction was 0.249164
product_id = 24851 - the prediction was 0.358929
product_id = 24963 - the prediction was 0.231816
product_id = 26208 - the predict

In [55]:
print(current_order_id)

44928


In [59]:
def calcF1(the_model, lowerbound, upperbound):
    
    # Get data
    y, x = getOrders(lowerbound, upperbound)
    
    # Initialize dictionary to hold actuals
    actual_dict = dict()
    
    y_to_user_and_number = dict()
    
    # Create dictionary for each order in y:
    for row in y:
        try:
            actual_dict[row[1]].add(row[3])
        except:
            actual_dict[row[1]] = set([row[3]])
            y_to_user_and_number[row[1]] = [row[0], row[3]]
     
    # for each order in actual_dict, go create and predict
    for order in list(actual_dict.keys()):
        
    
    return actual_dict
    #for product in [prod for prod in x_valid if prod[2] == final_order_number - j]:

In [60]:
calcF1(3, 70, 70)

{170: {5077,
  6236,
  8153,
  11182,
  13176,
  15093,
  17794,
  18394,
  21137,
  25591,
  34582,
  37766,
  39190,
  40354,
  43772,
  43841,
  49593},
 10270: {210, 8277, 29898, 37646, 39180, 39275, 42736, 45066},
 12770: {640,
  1405,
  2519,
  8584,
  19005,
  20691,
  27323,
  28079,
  30309,
  32605,
  42049,
  43715,
  44142,
  48705},
 14170: {863, 9561, 23339, 28698, 46313},
 14970: {25472, 42123, 43740},
 21270: {4210, 10108, 17794, 24852, 27921, 32624, 33109, 38400},
 23870: {1063,
  3479,
  14947,
  21137,
  21709,
  23543,
  26604,
  27966,
  37022,
  39275,
  41149,
  46206,
  48364},
 26070: {5161, 8174, 12606, 15937, 24852, 35725, 39947, 42265, 44683, 48679},
 36170: {432,
  9203,
  10493,
  12087,
  12312,
  12888,
  15070,
  16797,
  18210,
  21376,
  22247,
  24053,
  25232,
  26124,
  27138,
  32872,
  33043,
  37141,
  37465,
  41355,
  43631,
  45260},
 39970: {5876,
  9076,
  14069,
  18598,
  19162,
  24852,
  26283,
  26800,
  41149,
  45104,
  46979,
  4830